In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import json
import pandas as pd
import json
import yahoo_fin.stock_info as si

In [1]:
ticker = "AA"
url = "https://finance.yahoo.com/quote/{}/key-statistics?p={}"
response = requests.get(url.format(ticker, ticker))        
soup = bs(response.text, "html.parser")
pattern = re.compile(r"\s--\sData\s--\s")
script_data = soup.find("script", text = pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])
shares_outstanding = json_data["context"]["dispatcher"]["stores"]["QuoteSummaryStore"]["defaultKeyStatistics"]["sharesOutstanding"]["raw"]
shares_outstanding

NameError: name 'requests' is not defined

In [3]:
url = "https://finance.yahoo.com/quote/F/key-statistics?p=F"
response = requests.get(url.format(ticker, ticker))        
soup = bs(response.text, "html.parser")
pattern = re.compile(r"\s--\sData\s--\s")
script_data = soup.find("script", text = pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])
shares_outstanding = json_data["context"]["dispatcher"]["stores"]["QuoteSummaryStore"]["defaultKeyStatistics"]["bookValue"]["raw"]
shares_outstanding

8.326

### Try to webscrape history total assets, ebit, country

In [4]:
example = "https://finance.yahoo.com/quote/DAI.DE/profile?p=DAI.DE"

r = requests.get(example)

soup = bs(r.content)
info = soup.find_all("p", attrs = {"class", "D(ib) W(47.727%) Pend(40px)"})
info[0].text


'Mercedesstrasse 120Stuttgart 70372Germany49 711 170http://www.daimler.com'

In [5]:
countries = pd.read_excel("../countries.xlsx")

In [6]:
country_list = countries.name.to_list()

In [7]:
# save as json
with open("../countries.json", "w") as f:
    json.dump(country_list, f, indent=2)

In [8]:
for c in country_list:
    if c in info[0].text:
        print(c)
        break

Germany


## Assets

In [9]:
with open("../relevant_trading_212_tickers.json", "r") as f:
    relevant_trading_212_tickers = json.load(f)

In [ ]:
op_data = pd.DataFrame()

for ticker in relevant_trading_212_tickers[:1000]:
    try:
        

        ticker_df = pd.DataFrame({
            #"totalAssets","common_stock" : si.get_balance_sheet(ticker, yearly=False).loc[["totalAssets", "commonStock"]],
            "operating_income" : si.get_income_statement(ticker, yearly=False).loc["operatingIncome"],
            "ticker" : ticker
        })
        op_data = op_data.append(ticker_df)
    except:
        pass



In [ ]:
op_data

In [66]:
at_data = pd.DataFrame()

for ticker in relevant_trading_212_tickers[:1000]:
    try:
        

        ticker_df = pd.DataFrame({
            "totalAssets" : si.get_balance_sheet(ticker, yearly=False).loc["totalAssets"],
            "ticker" : ticker
        })
        at_data = at_data.append(ticker_df)
    except:
        pass

In [68]:
so_data = pd.DataFrame()

for ticker in relevant_trading_212_tickers[:1000]:
    try:
        

        ticker_df = pd.DataFrame({
            "common_stock" : si.get_balance_sheet(ticker, yearly=False).loc["commonStock"],
            "ticker" : ticker
        })
        so_data = so_data.append(ticker_df)
    except:
        pass
    
so_data

,common_stock,ticker
endDate,,
2020-09-30,3.513000e+06,EMCF
2020-06-30,3.513000e+06,EMCF
2020-03-31,3.513000e+06,EMCF
2019-12-31,3.513000e+06,EMCF
2020-09-30,3.800000e+04,AEIS
...,...,...
2019-12-31,4.005000e+09,E
2020-07-31,1.400000e+07,HPQ
2020-04-30,1.400000e+07,HPQ


In [69]:
len(relevant_trading_212_tickers)

4892